In [1]:
import os
import random
import pandas as pd
from PIL import Image
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

In [2]:
import sys
import gzip
import platform
import warnings
import collections
from tqdm import tqdm, tqdm_notebook
import matplotlib as mpl

In [3]:
# Set random seed

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED) # type : ignore
torch.backends.cudnn.deterministic = True # type : ignore
torch.backends.cudnn.benchmark = True # type : ignore


In [4]:
# 현재 OS 및 라이브러리 버전 체크 체크
current_os = platform.system()
print(f"Current OS: {current_os}")
print(f"CUDA: {torch.cuda.is_available()}")
print(f"Python Version: {platform.python_version()}")
print(f"torch Version: {torch.__version__}")
print(f"torchvision Version: {torchvision.__version__}")

# 중요하지 않은 에러 무시
warnings.filterwarnings(action='ignore')

# 유니코드 깨짐현상 해결
mpl.rcParams['axes.unicode_minus'] = False

Current OS: Linux
CUDA: True
Python Version: 3.8.5
torch Version: 1.10.2+cu102
torchvision Version: 0.11.3+cu102


In [5]:
image_path = []
image_label = []
image_id = []

for i in os.listdir('/opt/ml/input/data/train/label'):
    for j in os.listdir('/opt/ml/input/data/train/label/' + i):
        image_path.append('/opt/ml/input/data/train/label/' + i + '/' + j)
        image_label.append(int(i))
        image_id.append(str(j[:6]))

print(len(image_path), len(image_label))

train_path_label = pd.DataFrame()
train_path_label['id'] = image_id
train_path_label['path'] = image_path
train_path_label['label'] = image_label

print(train_path_label)

18900 18900
           id                                               path  label
0      003728  /opt/ml/input/data/train/label/11/003728_femal...     11
1      003442  /opt/ml/input/data/train/label/11/003442_femal...     11
2      005412  /opt/ml/input/data/train/label/11/005412_femal...     11
3      003731  /opt/ml/input/data/train/label/11/003731_femal...     11
4      003583  /opt/ml/input/data/train/label/11/003583_femal...     11
...       ...                                                ...    ...
18895  003840  /opt/ml/input/data/train/label/1/003840_male_A...      1
18896  003548  /opt/ml/input/data/train/label/1/003548_male_A...      1
18897  004446  /opt/ml/input/data/train/label/1/004446_male_A...      1
18898  005491  /opt/ml/input/data/train/label/1/005491_male_A...      1
18899  003488  /opt/ml/input/data/train/label/1/003488_male_A...      1

[18900 rows x 3 columns]


In [ ]:
# 사람별로 분류


In [6]:
# Dataset 생성

class CustomDataset(Dataset):
    def __init__(self, image_path_label, transform):
        self.X = image_path_label['path']
        self.y = image_path_label['label']
        self.transform = transform

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X, y = Image.open(self.X.iloc[idx]), self.y.iloc[idx]    
        
        if self.transform:
            X = self.transform(X)
        return X, torch.tensor(y)
    

In [7]:
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)), 
])

In [8]:
train_dateset = CustomDataset(train_path_label, transform)

In [29]:
train_path_label

,id,path,label
0,003728,/opt/ml/input/data/train/label/11/003728_femal...,11
1,003442,/opt/ml/input/data/train/label/11/003442_femal...,11
2,005412,/opt/ml/input/data/train/label/11/005412_femal...,11
3,003731,/opt/ml/input/data/train/label/11/003731_femal...,11
4,003583,/opt/ml/input/data/train/label/11/003583_femal...,11
...,...,...,...
18895,003840,/opt/ml/input/data/train/label/1/003840_male_A...,1
18896,003548,/opt/ml/input/data/train/label/1/003548_male_A...,1
18897,004446,/opt/ml/input/data/train/label/1/004446_male_A...,1
18898,005491,/opt/ml/input/data/train/label/1/005491_male_A...,1


In [9]:
BATCH_SIZE = 8
train_size = int(0.8 * len(train_path_label))
test_size = len(train_path_label) - train_size

train_dset, test_dset = random_split(train_dateset, [train_size, test_size], generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_dset, batch_size = BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(test_dset, batch_size=BATCH_SIZE, shuffle=True)

In [10]:
# batchsize, channel, height, width

next(iter(train_loader))[0].shape

torch.Size([8, 3, 512, 384])

In [11]:
efficientnet = torchvision.models.efficientnet_b5(pretrained = True)

In [12]:
import math

OUTPUT_CLASS_NUM = 18

efficientnet.fc = torch.nn.Linear(in_features=512, out_features=OUTPUT_CLASS_NUM, bias = True)

torch.nn.init.xavier_uniform_(efficientnet.fc.weight)
stdv = 1./math.sqrt(efficientnet.fc.weight.size(1))
efficientnet.fc.bias.data.uniform_(-stdv, stdv)

efficientnet.fc.weight.shape[0]

18

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [14]:
efficientnet.to(device)

LEARNING_RATE = 0.0001
NUM_EPOCH = 5

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(efficientnet.parameters(), lr = LEARNING_RATE)

dataloaders = {
    "train" : train_loader,
    "test" : valid_loader
}

In [15]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [16]:
best_test_accuracy = 0.
best_test_loss = 9999.

for epoch in range(NUM_EPOCH):
    for phase in ["train", "test"]:
        running_loss=  0.
        running_acc = 0.
        if phase == 'train':
            efficientnet.train()
        elif phase == 'test':
            efficientnet.eval()

        for ind, (images, labels) in enumerate(dataloaders[phase]):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == "train"):
                logits = efficientnet(images)
                _, preds = torch.max(logits, 1)
                loss = loss_fn(logits, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item()
            running_acc += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_acc / len(dataloaders[phase].dataset)

        print(f"현재 epoch.{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")

        if phase == "test" and best_test_loss < epoch_loss:
            best_test_accuracy = epoch_acc
        if phase == "test" and best_test_loss > epoch_loss:
            best_test_loss = epoch_loss

print("finished")
print(f"Best Accuracy : {best_test_accuracy}, Lowest Loss : {best_test_loss}")

현재 epoch.0의 train-데이터 셋에서 평균 Loss : 0.072, 평균 Accuracy : 0.842
현재 epoch.0의 test-데이터 셋에서 평균 Loss : 0.024, 평균 Accuracy : 0.931
현재 epoch.1의 train-데이터 셋에서 평균 Loss : 0.018, 평균 Accuracy : 0.953
현재 epoch.1의 test-데이터 셋에서 평균 Loss : 0.009, 평균 Accuracy : 0.974
현재 epoch.2의 train-데이터 셋에서 평균 Loss : 0.009, 평균 Accuracy : 0.978
현재 epoch.2의 test-데이터 셋에서 평균 Loss : 0.006, 평균 Accuracy : 0.986
현재 epoch.3의 train-데이터 셋에서 평균 Loss : 0.006, 평균 Accuracy : 0.985
현재 epoch.3의 test-데이터 셋에서 평균 Loss : 0.006, 평균 Accuracy : 0.987
현재 epoch.4의 train-데이터 셋에서 평균 Loss : 0.004, 평균 Accuracy : 0.990
현재 epoch.4의 test-데이터 셋에서 평균 Loss : 0.004, 평균 Accuracy : 0.991
finished
Best Accuracy : 0.0, Lowest Loss : 0.0038238652484203816


In [17]:
def func_eval(model, data_iter, device):
    with torch.no_grad():
        n_total, n_correct = 0, 0
        model.eval()
        for batch_in, batch_out in data_iter:
            y_trgt = batch_out.to(device)
            model_pred = model.forward(batch_in.to(device))
            _, y_pred = torch.max(model_pred, 1)
            n_correct += (y_pred == y_trgt).sum().item()
            n_total += batch_in.size(0)
        val_accr = (n_correct/n_total)
    return val_accr

In [18]:
func_eval(efficientnet, valid_loader, device)

0.9907407407407407

In [19]:
from pytz import timezone
import datetime as dt

now = (dt.datetime.now().astimezone(timezone("Asia/Seoul")).strftime("%Y-%m-%d-%H-%M"))

torch.save(model, f"save_model/{now}.pth")


0.9896825396825397